In [1]:
import nba_py
import nba_api
import numpy as np
import pandas as pd

In [2]:
#Get the Pacers team_id
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

# Select the dictionary for the Pacers, which contains their team ID
pacers = [team for team in nba_teams if team['abbreviation'] == 'IND'][0]
pacers_id = pacers['id']
print(f'pacers_id: {pacers_id}')

pacers_id: 1610612754


In [3]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=pacers_id,
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular)  

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')


Searching through 59 game(s) for the game_id of 0021800798 where IND vs. LAL


In [8]:
# Query for the play by play of that most recent regular season game
from nba_api.stats.endpoints import playbyplay
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
df.head() #just looking at the head of the data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0021800798,2,12,0,1,7:11 PM,12:00,None,None,None,None,None
1,0021800798,4,10,0,1,7:11 PM,12:00,Jump Ball Turner vs. Zubac: Tip to James,None,None,None,None
2,0021800798,7,2,75,1,7:11 PM,11:46,None,None,MISS Rondo 4' Driving Finger Roll Layup,None,None
3,0021800798,8,4,0,1,7:11 PM,11:42,Collison REBOUND (Off:0 Def:1),None,None,None,None
4,0021800798,9,1,98,1,7:12 PM,11:31,Bogdanovic 2' Cutting Layup Shot (2 PTS) (Young 1 AST),None,None,0 - 2,2


In [9]:
#Since the datset is fairly large you'll see plenty of elipses(...). 
#If that's the case, you can set the following options to expand the data 
#You can adjust these as you'd like
import pandas
pandas.set_option('display.max_colwidth',250)
pandas.set_option('display.max_rows',250)

In [10]:
#List unique values in the df['EVENTMSGTYPE'] colum
print(f'EVENTMSGTYPE: {sorted(df.EVENTMSGTYPE.unique())}')

EVENTMSGTYPE: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 18]


In [11]:
#For quick refernce, here's an Enum for `EVENTMSGTYPE`
#This list may be incomplete as a thourogh play by play scan is necessary

from enum import Enum

class EventMsgType(Enum):
    FIELD_GOAL_MADE = 1
    FIELD_GOAL_MISSED = 2
    FREE_THROWfree_throw_attempt = 3
    REBOUND = 4
    TURNOVER = 5
    FOUL = 6
    VIOLATION = 7
    SUBSTITUTION = 8
    TIMEOUT = 9
    JUMP_BALL = 10
    EJECTION = 11
    PERIOD_BEGIN = 12
    PERIOD_END = 13

In [12]:
#pull the data for a specfic EVENTMSGTYPE
df.loc[df['EVENTMSGTYPE'] == 2].head() #hint: use the EVENTMSGTYPE values above to see different data

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
2,0021800798,7,2,75,1,7:11 PM,11:46,None,None,MISS Rondo 4' Driving Finger Roll Layup,None,None
5,0021800798,11,2,1,1,7:12 PM,11:13,None,None,MISS Kuzma 26' 3PT Jump Shot,None,None
9,0021800798,19,2,1,1,7:13 PM,10:25,MISS Turner 16' Jump Shot,None,None,None,None
17,0021800798,31,2,6,1,7:17 PM,9:23,MISS Bogdanovic Driving Layup,None,None,None,None
28,0021800798,46,2,1,1,7:20 PM,8:29,None,None,MISS Kuzma 21' Jump Shot,None,None


In [13]:
#List unique values in the df['EVENTMSGTYPE'] column
emt_df = df.loc[df['EVENTMSGTYPE'] == 1]
print(f'EVENTMSGACTIONTYPE: {sorted(emt_df.EVENTMSGACTIONTYPE.unique())}')

EVENTMSGACTIONTYPE: [1, 5, 6, 7, 9, 43, 44, 47, 50, 52, 57, 58, 63, 66, 71, 72, 73, 75, 76, 79, 80, 86, 93, 97, 98, 103, 108]


In [14]:
#Mapping out all of the EventMsgActionTypes for EventMsgType 1
import re
import operator

#the following expression is specific to EventMsgType 1
p = re.compile('\s+([\w+ ]*)\s')

#get the PlayByPlay data from the Pacers game_id
plays = playbyplay.PlayByPlay(game_id).get_normalized_dict()['PlayByPlay']

#declare a few variables
description = ''
event_msg_action_types = {}

#loop over the play by play data
for play in plays:
    if play['EVENTMSGTYPE'] == 1:
        description = play['HOMEDESCRIPTION'] if not None else play['VISITORDESCRIPTION']
        if description is not None:
            #do a bit of searching(regex) and a little character magic: underscores and upper case
            event_msg_action = re.sub(' ', '_', p.search(description).groups()[0]).upper()
            #Add it to our dictionary
            event_msg_action_types[play['EVENTMSGACTIONTYPE']] = event_msg_action

#sort it all
event_msg_action_types = sorted(event_msg_action_types.items(), key=operator.itemgetter(0))

#output a class that we could plug into our code base
print('from enum import Enum\n')
print('class EventMsgActionType(Enum):')
for action in event_msg_action_types:
    print(f'\t{action[1]} = {action[0]}')

from enum import Enum

class EventMsgActionType(Enum):
	3PT_JUMP_SHOT = 1
	LAYUP = 5
	DUNK = 7
	REVERSE_LAYUP = 44
	DRIVING_HOOK_SHOT = 57
	TURNAROUND_HOOK_SHOT = 58
	JUMP_BANK_SHOT = 66
	FINGER_ROLL_LAYUP = 71
	PUTBACK_LAYUP = 72
	DRIVING_REVERSE_LAYUP = 73
	DRIVING_FINGER_ROLL_LAYUP = 75
	RUNNING_FINGER_ROLL_LAYUP = 76
	PULLUP_JUMP_SHOT = 79
	STEP_BACK_JUMP_SHOT = 80
	TURNAROUND_FADEAWAY = 86
	DRIVING_BANK_HOOK_SHOT = 93
	TIP_LAYUP_SHOT = 97
	CUTTING_LAYUP_SHOT = 98
	RUNNING = 103
	CUTTING_DUNK_SHOT = 108


In [15]:
#Mapping out all of the EventMsgActionTypes for EventMsgType 2
import re
import operator

#the following expression is specific to EventMsgType 1
p = re.compile('((?:MISS \S* \d*\')|(?:MISS \S*))\s*([\w+ ]*)')

#get the PlayByPlay data from the Pacers game_id
plays = playbyplay.PlayByPlay(game_id).get_normalized_dict()['PlayByPlay']

#declare a few variables
description = ''
event_msg_action_types = {}

#loop over the play by play data
#do a bit of findall(regex) and a little character magic: underscores and upper case
#we're using a findall here as we have to deal with the extra word MISS at the beginning of the text.
#that extra text means we'll have multiple matches for our regex.
for play in plays:
    if play['EVENTMSGTYPE'] == 2:
        match = list()
        if play['HOMEDESCRIPTION'] is not None: 
            match = p.findall(play['HOMEDESCRIPTION'])
        
        if not match:
            match = p.findall(play['VISITORDESCRIPTION'])

        event_msg_action = re.sub(' ', '_', match[0][1]).upper()
        event_msg_action_types[play['EVENTMSGACTIONTYPE']] = event_msg_action
            
event_msg_action_types = sorted(event_msg_action_types.items(), key=operator.itemgetter(0))

print('from enum import Enum\n')
print('class EventmsgActionType(Enum):')
for action in event_msg_action_types:
    print(f'\t{action[1]} = {action[0]}')

from enum import Enum

class EventmsgActionType(Enum):
	3PT_JUMP_SHOT = 1
	HOOK_SHOT = 3
	LAYUP = 5
	DRIVING_LAYUP = 6
	REVERSE_LAYUP = 44
	TURNAROUND_JUMP_SHOT = 47
	ALLEY_OOP_DUNK = 52
	TURNAROUND_HOOK_SHOT = 58
	FINGER_ROLL_LAYUP = 71
	PUTBACK_LAYUP = 72
	DRIVING_REVERSE_LAYUP = 73
	DRIVING_FINGER_ROLL_LAYUP = 75
	FLOATING_JUMP_SHOT = 78
	3PT_PULLUP_JUMP_SHOT = 79
	STEP_BACK_JUMP_SHOT = 80
	TURNAROUND_FADEAWAY_SHOT = 86
	TIP_LAYUP_SHOT = 97
	DRIVING_FLOATING_JUMP_SHOT = 101
	DRIVING_FLOATING_BANK_JUMP_SHOT = 102
	TURNAROUND_FADEAWAY_BANK_JUMP_SHOT = 105
	CUTTING_DUNK_SHOT = 108
